# 멜론 음원 차트 크롤링

## 연도별

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re
import pandas as pd

# 웹 드라이버 실행
driver = webdriver.Chrome('chromedriver.exe', options=options)
url = "https://www.melon.com/chart/index.htm"

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36'}

song_data = []
year = 2020

# 차트 파인더 조건 선택
for i in range(1,3):  # 연대 2020년대, 2019년대
    for j in range(1,4):  # 연도 2017년까지
        driver.get(url)
        time.sleep(2)
        driver.find_elements_by_css_selector('li.nth1.on > div > div > button')[0].click()  # 차트 파인더 페이지로 이동
        time.sleep(2)
        driver.find_elements_by_css_selector('h4.tab03 > a')[0].click()  # 차트 선택
        time.sleep(2)
        driver.find_elements_by_css_selector(f'div.box_chic.nth1.view > div.list_value > ul > li:nth-child({i})')[0].click()  # 연대 선택
        time.sleep(2)
        try:
            driver.find_elements_by_css_selector(f'div.box_chic.nth2.view > div.list_value > ul > li:nth-child({j})')[0].click()  # 연도 선택
            time.sleep(2)
        except IndexError :
            continue
        driver.find_elements_by_css_selector('div.box_chic.last.view > div.list_value > ul > li:nth-child(2)')[0].click()  # 장르/스타일 선택
        time.sleep(2)
        driver.find_elements_by_css_selector('div.wrap_btn_serch > button')[0].click()  # 검색 버튼 클릭
        time.sleep(2)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        songs = soup.select('table > tbody > tr')
        rank = 1

        for song in songs:
            title = song.select_one('div.rank01 > span > strong > a').text  # 제목 추출
            singer = song.select_one('div.rank02 > a').text  # 가수 추출
            songid = song.select_one('td:nth-of-type(1) > div > input')['value']  # 곡 고유번호

            song_url = f'https://www.melon.com/song/detail.htm?songId={songid}'  # 곡 정보 페이지 url
            song_html = requests.get(song_url, headers=headers).text
            song_soup = BeautifulSoup(song_html, 'html.parser')

            genre = song_soup.select_one('div.meta > dl > dd:nth-of-type(6)')  # 장르 추출

            song_data.append([year, rank, title, singer, genre])  # 추출한 데이터 리스트로 저장
            rank += 1
        year -= 1


# # 데이터 프레임으로 만든 후 엑셀파일로 저장
columns = ['년도', '순위', '타이틀', '가수', '장르']
pd_data = pd.DataFrame(song_data, columns = columns)
pd_data.to_excel('./data/melon_year.xlsx', index=False)